# データサイエンス100本ノック（構造化データ加工編） - Python

はじめに
初めに以下のセルを実行してください
必要なライブラリのインポートとデータベース（PostgreSQL）からのデータ読み込みを行います
pandas等、利用が想定されるライブラリは以下セルでインポートしています
その他利用したいライブラリがあれば適宜インストールしてください（"!pip install ライブラリ名"でインストールも可能）
処理は複数回に分けても構いません
名前、住所等はダミーデータであり、実在するものではありません

In [2]:
import os
import pandas as pd
import numpy as np
from datetime import datetime, date
from dateutil.relativedelta import relativedelta
import math
import psycopg2
from sqlalchemy import create_engine
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from imblearn.under_sampling import RandomUnderSampler # conda install -c conda-forge imbalanced-learn

df_customer = pd.read_csv("./data/customer.csv")
df_category = pd.read_csv("./data/category.csv")
df_product = pd.read_csv("./data/product.csv")
df_receipt = pd.read_csv("./data/receipt.csv")
df_store = pd.read_csv("./data/store.csv")
df_geocode = pd.read_csv("./data/geocode.csv")

C:\Users\335395\AppData\Local\Continuum\anaconda3\envs\Yugo environment\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


P-021: レシート明細データフレーム（df_receipt）に対し、件数をカウントせよ。

In [3]:
df_receipt.head()

,sales_ymd,sales_epoch,store_cd,receipt_no,receipt_sub_no,customer_id,product_cd,quantity,amount
0,20181103,1257206400,S14006,112,1,CS006214000001,P070305012,1,158
1,20181118,1258502400,S13008,1132,2,CS008415000097,P070701017,1,81
2,20170712,1215820800,S14028,1102,1,CS028414000014,P060101005,1,170
3,20190205,1265328000,S14042,1132,1,ZZ000000000000,P050301001,1,25
4,20180821,1250812800,S14025,1102,2,CS025415000050,P060102007,1,90


In [7]:
df_receipt.count()

sales_ymd         104681
sales_epoch       104681
store_cd          104681
receipt_no        104681
receipt_sub_no    104681
customer_id       104681
product_cd        104681
quantity          104681
amount            104681
dtype: int64

In [13]:
len(df_receipt)

104681

P-022: レシート明細データフレーム（df_receipt）の顧客ID（customer_id）に対し、ユニーク件数をカウントせよ。

In [11]:
len(df_receipt['customer_id'].unique())

8307

P-023: レシート明細データフレーム（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）と売上数量（quantity）を合計せよ。

In [34]:
df1 = df_receipt.groupby('store_cd').sum()
df1 = df1[['amount','quantity']]
df1

,amount,quantity
store_cd,,
S12007,638761,2099
S12013,787513,2425
S12014,725167,2358
S12029,794741,2555
S12030,684402,2403
S13001,811936,2347
S13002,727821,2340
S13003,764294,2197
S13004,779373,2390


In [33]:
#解2
df_receipt.groupby('store_cd').agg({'amount':'sum', 'quantity':'sum'})

,amount,quantity
store_cd,,
S12007,638761,2099
S12013,787513,2425
S12014,725167,2358
S12029,794741,2555
S12030,684402,2403
S13001,811936,2347
S13002,727821,2340
S13003,764294,2197
S13004,779373,2390


P-024: レシート明細データフレーム（df_receipt）に対し、顧客ID（customer_id）ごとに最も新しい売上日（sales_ymd）を求め、10件表示せよ。

In [39]:
df2 = df_receipt.groupby('customer_id').max()

In [40]:
df2 = df2['sales_ymd']
df2.head(10)

customer_id
CS001113000004    20190308
CS001114000005    20190731
CS001115000010    20190405
CS001205000004    20190625
CS001205000006    20190224
CS001211000025    20190322
CS001212000027    20170127
CS001212000031    20180906
CS001212000046    20170811
CS001212000070    20191018
Name: sales_ymd, dtype: int64

In [41]:
#解2
df_receipt.groupby('customer_id').sales_ymd.max().reset_index().head(10)

,customer_id,sales_ymd
0,CS001113000004,20190308
1,CS001114000005,20190731
2,CS001115000010,20190405
3,CS001205000004,20190625
4,CS001205000006,20190224
5,CS001211000025,20190322
6,CS001212000027,20170127
7,CS001212000031,20180906
8,CS001212000046,20170811
9,CS001212000070,20191018


P-025: レシート明細データフレーム（df_receipt）に対し、顧客ID（customer_id）ごとに最も古い売上日（sales_ymd）を求め、10件表示せよ。

In [30]:
df3 = df_receipt.groupby('customer_id').min()

In [31]:
df3.head(10)

,sales_ymd,sales_epoch,store_cd,receipt_no,receipt_sub_no,product_cd,quantity,amount
customer_id,,,,,,,,
CS001113000004,20190308,1268006400,S13001,112,1,P071001012,1,98
CS001114000005,20180503,1241308800,S13001,112,1,P050101001,1,40
CS001115000010,20171228,1230422400,S13001,112,1,P070101003,1,48
CS001205000004,20170914,1221350400,S13001,1132,1,P040501001,1,40
CS001205000006,20180207,1233964800,S13001,1152,1,P040101003,1,30
CS001211000025,20190322,1269216000,S13001,112,1,P050601002,1,158
CS001212000027,20170127,1201392000,S13001,112,1,P040403002,1,200
CS001212000031,20180906,1252195200,S13001,112,1,P070203012,1,148
CS001212000046,20170811,1218412800,S13001,112,1,P071006001,1,108
